## 매개변수 갱신 

신경망 학습의 목적은 손실 함수의 값을 가능한 낮추는 매개변수를 찾는 것이다. 이는 곧 매개변수의 최적값을 찾는 문제이며, 이러한 문제를 푸는 것을 최적화라고 한다. 



## 확률적 경사 하강법 (SGD) 

<img src="https://t1.daumcdn.net/cfile/tistory/9978B5465AA8029F22" width="200" height="300"/> 

W는 갱신할 가중치 매개변수이고 미분값은 W에 대한 손실 함수의 기울기 이다. n는 학습률의 의미한다. <-는 우변의 값으로 좌변의 값을 갱신한다는 뜻이다. 





In [1]:
class SGD :
    def __init__ (self, lr = 0.01)  :
        self.lr = lr
        
    def update (self, params, grads) :
        for k in params.keys() :
            params[k] -= self.lr * grads[k ]

### SGD 의 단점 

SGD는 단순하고 구현도 쉽지만, 문제에 따라서는 비효율적일 때가 있다. SGD는 비등방성(anisotropy)함수이기 때문이다. 방향에 따라 성질, 즉 기울기가 달라진다는 의미이다. SGD가 지그재그로 탐색하는 근본 원인은 기울어진 방향이 본래의 최솟값과 다른 방향을 가리켜서라는 점도 고려해봐야 한다. 

## 모멘텀(Momentum) 

<img src="https://blog.kakaocdn.net/dn/bmYcIj/btqJZopKIwQ/BTg82XwDLnVH2rsxZBnXEK/img.png" width="200" height="300"/> 

다른 변수들은 SGD와 동일하고 v라는 변수는 물리에서 말하는 속도에 해당한다. 기울기 방향으로 힘을 받아 물체가 가속된다는 물리법칙을 나타낸다.  
av항은 물체가 아무런 힘을 받지 않은 때 서서히 하강 시키는 역할을 하는 변수이다. 물리에서의 지면 마찰이나 공기 저항에 해당된다. 

In [2]:
class Momentum :
    def __init__ (self, lr = 0.01, m = 0.9) :
        self.lr = lr
        self.m = m
        self.v = None
        
    def updare(self, params, grads) :
        if self.v is None :
            self.v = {}
            for k , val in params.items() :
                self.v[k] = np.zeros_like(val)
                
        for k in params.keys() :
            self.v[k] = self.m * self.v[k] - self.lr * grads[k]
            params[k] += self.v[k]

모멘텀는 지그재그 정도가 덜하다. 이는 x축의 힘은 아주 작지만 방향은 변하지 않아서 한 방향으로 일정하게 가속하기 때문이다. 거꾸로 y축은 힘은 크지만 위아래로 번갈아 힘을 받아 상충하게 되며 y축 방향의 속도는 안정적이지 않다. 

## AdaGrad 

신경망 학습에서는 학습률 값이 중요하다. 이 값이 너무 작으면 학습 시간이 너무 길어지고, 반대로 너무 크면 학습이 제대로 이루어지지 않는다. 

이 학습률을 정하는 효과적 기술로 **학습률 감소,learning rate decay**가 있다. 이는 학습을 진행하면서 학습률을 점차 줄여가는 방법이다. 처음에는 크게 학습하다가 조금씩 작게 학습한다는 의미로, 실제 신경망 학습에 자주 쓰인다. 

학습률을 서서히 낮추는 가장 간단한 방법은 매개변수'전체'의 학습률 값을 일괄적으로 낮추는 것이다. 이를 더욱 발전시킨 것이 **AdaGrad**이다. AdaGrad는 각각의 매매변수에 맞춤형 값을 만들어 준다. 

----- 

AdaGrad는 개별 매개변수에 적응적으로 학습률을 조정하면서 학습을 진행한다. 
<img src="https://blog.kakaocdn.net/dn/H8MY9/btqJ0owhSUL/j7keQZ0PVQfy7RMS9F9tyK/img.png" width="200" height="300"/> 
마찬가지로 W는 갱신할 가중치 매개변수, 편미분 값은 W에 대한 손실함수의 기울기, n는 학습률을 의미한다. h라는 변수가 나오는데 h는 기존 기울기 값을 제곱하여 계속 더해준다. 그리고 매개변수를 갱신할 때 $ 1/\sqrt{h}$ 을 곱해 학습률을 조정한다. 매개변수의 원소 중에서 많이 움직인 원소를 학습률이 낮아진다는 뜻인데, **학습률 감소가 매개변수의 원소마다 다르게 적용됨을 의미한다.** 

-------

AdaGrad는 과거의 기울기를 제곱하여 계속 더해간다. 학습을 진행할 수록 갱신 강도가 약해진다. 실제로 무한히 계속 학습한다면 어느 순간 갱신량이 0이 되어 전혀 갱신되지 않게 된다. 이 문제를 개선한 기법으로 RMSProp이라는 방법이 존재한다. RMSProp은 과거의 모든 기울기를 균일하게 더해가는 것이 아니라, 먼 과거의 기울기를 서서히 잊고 새로운 기울기 정보를 크게 반영한다. 이를 **지수이동평균**이라 하여, 과거 기울기의 반영 규모를 기하급수적으로 감소 시킨다. 

In [3]:
class AdaGrad :
    def __init__(self, lr = 0.01) :
        self.lr =lr 
        self.h = None
        
    def update (self, params, grads) :
        if self.h in None :
            self.h = {} 
            for k , v in params, items () :
                self.h[k] = np.zeros_like(v)
                
        for k in params.keys () :
            self.h[k] += grads[k] + grads[k]
            params[k] -= self.lr * grads[k] / (np.sqrt(self.h[k]) + 1e-7)            